<a href="https://colab.research.google.com/github/gez2code/DeepUnderstandingOfDeepLearning/blob/main/main_experiment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# 1. Install the necessary libraries
# - wandb: for experiment tracking
# - medmnist: for the dataset
!pip install -q wandb medmnist

# 2. Import libraries
import wandb
from google.colab import userdata # This reads the secret key
import os

# 3. Login to WandB securely
# This grabs the key you saved in the "Secrets" tab
wandb_key = userdata.get('WANDB_API_KEY')
wandb.login(key=wandb_key)

# 4. Initialize the Project
# This creates a project in your WandB dashboard
run = wandb.init(
    project="DermaMNIST-Hybrid-Project",
    name="Setup-Test-Run",
    notes="Testing if the lab environment works."
)

# 5. Log a dummy metric to make sure it works
wandb.log({"accuracy": 0.5, "loss": 1.2})
print("✅ Setup Complete! Check your WandB dashboard.")

# 6. Finish the run
wandb.finish()

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.9/115.9 kB 3.4 MB/s eta 0:00:00


/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: abraham-gezehei (abraham-gezehei-fachhochschule-nordwestschweiz-fhnw) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


✅ Setup Complete! Check your WandB dashboard.


accuracy,▁
loss,▁
accuracy,0.5
loss,1.2


In [2]:
# ==========================================
# PHASE 2: DATA PIPELINE
# ==========================================
import torch
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from medmnist import INFO, Evaluator
import medmnist

# 1. Define the Data Configuration
# We use 'dermamnist' as requested by the professor
data_flag = 'dermamnist'
info = INFO[data_flag]
n_channels = info['n_channels']
n_classes = len(info['label'])
DataClass = getattr(medmnist, info['python_class'])

print(f"📊 Dataset Selected: {data_flag}")
print(f"ℹ️  Channels: {n_channels} | Classes: {n_classes}")

# 2. Preprocessing (The "Transform")
# We convert images to Tensors (numbers) and Normalize them
# Normalization (mean=0.5, std=0.5) helps the neural network learn faster
data_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[.5], std=[.5])
])

# 3. Download and Load the Data
# 'download=True' ensures you get the files automatically
train_dataset = DataClass(split='train', transform=data_transform, download=True)
val_dataset = DataClass(split='val', transform=data_transform, download=True)
test_dataset = DataClass(split='test', transform=data_transform, download=True)

# 4. Create "Data Loaders"
# These shuffle the data and feed it to the GPU in batches of 128 images at a time
BATCH_SIZE = 128
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

# 5. SANITY CHECK (Crucial Step!)
# Always check the shape of your data before training
images, labels = next(iter(train_loader))

print("\n✅ Data Loading Successful!")
print(f"📦 Batch Shape: {images.shape}")
print(f"   -> This means: [Batch_Size, Channels, Height, Width]")
print(f"🏷️ Labels Shape: {labels.shape}")

📊 Dataset Selected: dermamnist
ℹ️  Channels: 3 | Classes: 7


100%|██████████| 19.7M/19.7M [00:44<00:00, 443kB/s]



✅ Data Loading Successful!
📦 Batch Shape: torch.Size([128, 3, 28, 28])
   -> This means: [Batch_Size, Channels, Height, Width]
🏷️ Labels Shape: torch.Size([128, 1])


In [3]:
# ==========================================
# PHASE 3.1: THE TRAINING ENGINE
# ==========================================
import torch.optim as optim
import torch.nn as nn

# Check device again to be safe
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"⚙️ Engine running on: {device}")

def train_engine(model, model_name, epochs=10):
    """
    Args:
        model: The neural network to train
        model_name: String name for the report (e.g. "Baseline_CNN")
        epochs: How many times to loop through the data
    """

    # 1. Initialize WandB for this specific run
    wandb.init(
        project="DermaMNIST-Hybrid-Project",
        name=f"Run_{model_name}",
        config={
            "architecture": model_name,
            "dataset": "DermaMNIST",
            "epochs": epochs,
            "batch_size": 128,
            "learning_rate": 0.001
        },
        reinit=True # Allows multiple runs in one notebook
    )

    # 2. Setup Optimizer & Loss Function
    model = model.to(device)
    criterion = nn.CrossEntropyLoss() # Standard for classification
    optimizer = optim.Adam(model.parameters(), lr=0.001)

    print(f"\n🚀 Starting training for: {model_name}")

    # 3. The Training Loop
    for epoch in range(epochs):
        model.train() # Switch to training mode
        running_loss = 0.0
        correct = 0
        total = 0

        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            # labels coming from MedMNIST might be [Batch, 1], we need [Batch]
            labels = labels.squeeze().long()

            # Zero gradients
            optimizer.zero_grad()

            # Forward pass
            outputs = model(inputs)
            loss = criterion(outputs, labels)

            # Backward pass (Learning)
            loss.backward()
            optimizer.step()

            # Metrics
            running_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

        # Calculate average metrics for this epoch
        epoch_loss = running_loss / len(train_loader)
        epoch_acc = correct / total

        # Log to WandB (The "Lab Notebook")
        wandb.log({"epoch": epoch, "train_loss": epoch_loss, "train_acc": epoch_acc})
        print(f"   [Epoch {epoch+1}/{epochs}] Loss: {epoch_loss:.4f} | Acc: {epoch_acc:.4f}")

    # 4. Final Evaluation on Test Set
    print(f"📝 Evaluating {model_name} on Test Set...")
    model.eval() # Switch to eval mode (freezes layers like Dropout)
    correct = 0
    total = 0

    with torch.no_grad(): # No need to calculate gradients for testing
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            labels = labels.squeeze().long()
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    test_acc = correct / total
    print(f"🏆 Final Test Accuracy for {model_name}: {test_acc:.4f}")

    # Log final score to WandB
    wandb.log({"test_accuracy": test_acc})
    wandb.finish()

    return model, test_acc

⚙️ Engine running on: cpu


In [4]:
# ==========================================
# PHASE 3.2: THE BASELINE MODEL
# ==========================================

class SimpleCNN(nn.Module):
    def __init__(self, num_classes):
        super(SimpleCNN, self).__init__()
        # Layer 1: Conv -> BatchNorm -> ReLU -> MaxPool
        self.layer1 = nn.Sequential(
            nn.Conv2d(3, 16, kernel_size=3, padding=1),
            nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        # Layer 2: Conv -> BatchNorm -> ReLU -> MaxPool
        self.layer2 = nn.Sequential(
            nn.Conv2d(16, 32, kernel_size=3, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        # Fully Connected Layer (Classifier)
        # Image is 28x28. After two MaxPools (divide by 2 twice), it is 7x7.
        # So input size is 32 channels * 7 * 7
        self.fc = nn.Linear(32 * 7 * 7, num_classes)

    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.view(out.size(0), -1) # Flatten the image into a vector
        out = self.fc(out)
        return out

# --- RUN THE EXPERIMENT ---
# 1. Instantiate the model
baseline_model = SimpleCNN(num_classes=n_classes)

# 2. Train it using our Engine
trained_baseline, baseline_acc = train_engine(baseline_model, "Baseline_CNN", epochs=10)

wandb: WARNING Using a boolean value for 'reinit' is deprecated. Use 'return_previous' or 'finish_previous' instead.



🚀 Starting training for: Baseline_CNN
   [Epoch 1/10] Loss: 0.9316 | Acc: 0.6775
   [Epoch 2/10] Loss: 0.7763 | Acc: 0.7206
   [Epoch 3/10] Loss: 0.7183 | Acc: 0.7378
   [Epoch 4/10] Loss: 0.6867 | Acc: 0.7488
   [Epoch 5/10] Loss: 0.6570 | Acc: 0.7612
   [Epoch 6/10] Loss: 0.6352 | Acc: 0.7688
   [Epoch 7/10] Loss: 0.6125 | Acc: 0.7722
   [Epoch 8/10] Loss: 0.6028 | Acc: 0.7749
   [Epoch 9/10] Loss: 0.5843 | Acc: 0.7851
   [Epoch 10/10] Loss: 0.5700 | Acc: 0.7885
📝 Evaluating Baseline_CNN on Test Set...
🏆 Final Test Accuracy for Baseline_CNN: 0.7387


epoch,▁▂▃▃▄▅▆▆▇█
test_accuracy,▁
train_acc,▁▄▅▅▆▇▇▇██
train_loss,█▅▄▃▃▂▂▂▁▁
epoch,9
test_accuracy,0.73865
train_acc,0.7885
train_loss,0.57004


In [ ]:
# ==========================================
# PHASE 3.3: THE COMPETITOR (ResNet-18)
# ==========================================
from torchvision.models import resnet18

def get_resnet_for_small_images(num_classes):
    # 1. Load standard ResNet-18 Structure
    # weights=None means we train from scratch (fair comparison with Baseline)
    model = resnet18(weights=None)

    # 2. THE HACK (Critical for MedMNIST)
    # Original ResNet starts with an aggressive 7x7 conv and 2x2 pooling.
    # We replace it with a gentle 3x3 conv and NO pooling to preserve our small 28x28 images.
    model.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False)
    model.maxpool = nn.Identity() # Remove the first maxpool

    # 3. Modify the final classifier
    # ResNet outputs 512 features, we need 'num_classes' outputs
    model.fc = nn.Linear(512, num_classes)

    return model

# --- RUN THE EXPERIMENT ---
print("🏗️ Building ResNet-18 Competitor...")
competitor_model = get_resnet_for_small_images(num_classes=n_classes)

# 2. Train using the SAME engine (for fair comparison)
# Note: This will take longer because ResNet is much deeper!
trained_competitor, competitor_acc = train_engine(competitor_model, "ResNet18_SOTA", epochs=10)

🏗️ Building ResNet-18 Competitor...



🚀 Starting training for: ResNet18_SOTA
   [Epoch 1/10] Loss: 0.9336 | Acc: 0.6689
   [Epoch 2/10] Loss: 0.7911 | Acc: 0.7087
   [Epoch 3/10] Loss: 0.7476 | Acc: 0.7196
   [Epoch 4/10] Loss: 0.6932 | Acc: 0.7408
   [Epoch 5/10] Loss: 0.6733 | Acc: 0.7488
